In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt

from tf_seg.deploy import Inferencer

In [ ]:
inference_config = dict(
    image_size=(512, 512),
    normalizing=True,
    model_type="onnx",  # onnx or tf
    model_path="test_resnet50_binary_road",  # model folder path , not file
    preprocessor_type="albumentations",
    preprocessor_path="tf_seg.transformers.albumentations:get_test_transform",
    postprocessor_type="vanilla", # vanilla or None
    postprocessor_path=None,
    threshold=0.9,
    seed=48,
    device="gpu" # cpu or gpu
)

In [ ]:
inf = Inferencer(**inference_config)

In [ ]:
image_path1 = "test_images/road_1.webp"
image_path2 = "test_images/road_2.jpg"

In [ ]:
image = plt.imread(image_path2)
image.shape

In [ ]:
pred = inf.predict(image)

In [ ]:
pred_image = pred[1][0]
plt.imshow(pred_image)

In [ ]:
plt.imshow(inf.input_image[0])

In [ ]:
%%timeit
pred = inf.predict(image)

In [ ]:
compare_list = {
    "onnx_cpu": {"device": "cpu", "model_type": "onnx"},
    "onnx_gpu": {"device": "gpu", "model_type": "onnx"},
    "tf_cpu": {"device": "cpu", "model_type": "tf"},
    "tf_gpu": {"device": "gpu", "model_type": "tf"},
}


model_list = {"restnet": {"model_path": "test_resnet50_binary_road"}, "efficientnet": {"model_path": "test_efficientnetb0_binary_road"}}


In [ ]:
for model_name,model_path in model_list.items():
    for key, value in compare_list.items():
        print(model_name+"_"+key)
        inference_config.update(value)
        inference_config.update(model_path)
        inf = Inferencer(**inference_config)
        %timeit pred = inf.predict(image)
        print("-"*50)